In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("interworks").getOrCreate()

In [2]:
flights_df = spark.read.format("csv")\
  .option("header", "true")\
  .option("delimiter","|")\
  .option("encoding", "UTF-8")\
  .load("flights.txt")
flights_df.printSchema()
#flights_df.describe()

root
 |-- TRANSACTIONID: string (nullable = true)
 |-- FLIGHTDATE: string (nullable = true)
 |-- AIRLINECODE: string (nullable = true)
 |-- AIRLINENAME: string (nullable = true)
 |-- TAILNUM: string (nullable = true)
 |-- FLIGHTNUM: string (nullable = true)
 |-- ORIGINAIRPORTCODE: string (nullable = true)
 |-- ORIGAIRPORTNAME: string (nullable = true)
 |-- ORIGINCITYNAME: string (nullable = true)
 |-- ORIGINSTATE: string (nullable = true)
 |-- ORIGINSTATENAME: string (nullable = true)
 |-- DESTAIRPORTCODE: string (nullable = true)
 |-- DESTAIRPORTNAME: string (nullable = true)
 |-- DESTCITYNAME: string (nullable = true)
 |-- DESTSTATE: string (nullable = true)
 |-- DESTSTATENAME: string (nullable = true)
 |-- CRSDEPTIME: string (nullable = true)
 |-- DEPTIME: string (nullable = true)
 |-- DEPDELAY: string (nullable = true)
 |-- TAXIOUT: string (nullable = true)
 |-- WHEELSOFF: string (nullable = true)
 |-- WHEELSON: string (nullable = true)
 |-- TAXIIN: string (nullable = true)
 |-- CR

In [3]:
flights_df.show(1)

+-------------+----------+-----------+--------------------+-------+---------+-----------------+--------------------+--------------+-----------+---------------+---------------+--------------------+------------+---------+-------------+----------+-------+--------+-------+---------+--------+------+----------+-------+--------+--------------+-----------------+---------+--------+---------+
|TRANSACTIONID|FLIGHTDATE|AIRLINECODE|         AIRLINENAME|TAILNUM|FLIGHTNUM|ORIGINAIRPORTCODE|     ORIGAIRPORTNAME|ORIGINCITYNAME|ORIGINSTATE|ORIGINSTATENAME|DESTAIRPORTCODE|     DESTAIRPORTNAME|DESTCITYNAME|DESTSTATE|DESTSTATENAME|CRSDEPTIME|DEPTIME|DEPDELAY|TAXIOUT|WHEELSOFF|WHEELSON|TAXIIN|CRSARRTIME|ARRTIME|ARRDELAY|CRSELAPSEDTIME|ACTUALELAPSEDTIME|CANCELLED|DIVERTED| DISTANCE|
+-------------+----------+-----------+--------------------+-------+---------+-----------------+--------------------+--------------+-----------+---------------+---------------+--------------------+------------+---------+---------

In [4]:
# Miles data
flights_df.createOrReplaceTempView("flights")
miles_sql = "select distance from flights"
sql = "select count(*) from flights"
count = spark.sql(sql)
#count.show()

sql = "select count(*) from flights where distance like '%miles%'"
count2 = spark.sql(sql)
#count2.show()

sql = """
select TRANSACTIONID,FLIGHTDATE,AIRLINECODE,AIRLINENAME,TAILNUM,FLIGHTNUM,ORIGINAIRPORTCODE,
ORIGAIRPORTNAME,ORIGINCITYNAME,ORIGINSTATE,ORIGINSTATENAME,DESTAIRPORTCODE,DESTAIRPORTNAME,
DESTCITYNAME,DESTSTATE,DESTSTATENAME,CRSDEPTIME,DEPTIME,DEPDELAY,TAXIOUT,WHEELSOFF,WHEELSON,TAXIIN,
CRSARRTIME,ARRTIME,ARRDELAY,CRSELAPSEDTIME,ACTUALELAPSEDTIME,CANCELLED,DIVERTED,DISTANCE,
replace(distance , 'miles' ,'') as distance_mod 
from flights 
"""
miles = spark.sql (sql)
#miles.show(2)
miles.createOrReplaceTempView("miles")


sql = """select 
TRANSACTIONID,FLIGHTDATE,AIRLINECODE,AIRLINENAME,TAILNUM,FLIGHTNUM,ORIGINAIRPORTCODE,
ORIGAIRPORTNAME,ORIGINCITYNAME,ORIGINSTATE,ORIGINSTATENAME,DESTAIRPORTCODE,DESTAIRPORTNAME,
DESTCITYNAME,DESTSTATE,DESTSTATENAME,CRSDEPTIME,DEPTIME,DEPDELAY,TAXIOUT,WHEELSOFF,WHEELSON,TAXIIN,
CRSARRTIME,ARRTIME,ARRDELAY,CRSELAPSEDTIME,ACTUALELAPSEDTIME,CANCELLED,DIVERTED,DISTANCE,
distance_mod ,
Floor((distance_mod - 0)/100 ) as bin  , 
Floor((distance_mod - 0)/100 )*100 start_bin, 
(Floor((distance_mod - 0)/100 )+1)*100 end_bin,
concat(Floor((distance_mod - 0)/100 )*100 , '-',(Floor((distance_mod - 0)/100 )+1)*100, ' miles') DISTANCEGROUP
from miles 

"""
dg_miles = spark.sql (sql)
dg_miles.show(1)

+-------------+----------+-----------+--------------------+-------+---------+-----------------+--------------------+--------------+-----------+---------------+---------------+--------------------+------------+---------+-------------+----------+-------+--------+-------+---------+--------+------+----------+-------+--------+--------------+-----------------+---------+--------+---------+------------+---+---------+-------+-------------+
|TRANSACTIONID|FLIGHTDATE|AIRLINECODE|         AIRLINENAME|TAILNUM|FLIGHTNUM|ORIGINAIRPORTCODE|     ORIGAIRPORTNAME|ORIGINCITYNAME|ORIGINSTATE|ORIGINSTATENAME|DESTAIRPORTCODE|     DESTAIRPORTNAME|DESTCITYNAME|DESTSTATE|DESTSTATENAME|CRSDEPTIME|DEPTIME|DEPDELAY|TAXIOUT|WHEELSOFF|WHEELSON|TAXIIN|CRSARRTIME|ARRTIME|ARRDELAY|CRSELAPSEDTIME|ACTUALELAPSEDTIME|CANCELLED|DIVERTED| DISTANCE|distance_mod|bin|start_bin|end_bin|DISTANCEGROUP|
+-------------+----------+-----------+--------------------+-------+---------+-----------------+--------------------+--------------

In [5]:
dg_miles.createOrReplaceTempView("dg_miles")

sql = """
select 
TRANSACTIONID,FLIGHTDATE,AIRLINECODE,AIRLINENAME,TAILNUM,FLIGHTNUM,ORIGINAIRPORTCODE,
ORIGAIRPORTNAME,ORIGINCITYNAME,ORIGINSTATE,ORIGINSTATENAME,DESTAIRPORTCODE,DESTAIRPORTNAME,
DESTCITYNAME,DESTSTATE,DESTSTATENAME,CRSDEPTIME,DEPTIME,TAXIOUT,WHEELSOFF,WHEELSON,TAXIIN,
CRSARRTIME,ARRTIME,ARRDELAY,CRSELAPSEDTIME,ACTUALELAPSEDTIME,CANCELLED,DIVERTED,DISTANCE,
distance_mod ,
bin,start_bin,end_bin,DISTANCEGROUP,
DEPDELAY ,
case 
  WHEN DEPDELAY > 15 THEN 1 
  ELSE 0 
END as DEPDELAYGT15   
from dg_miles
"""
delay_miles = spark.sql (sql)
delay_miles.show(2)


# Negative values in Departure delay . Assumption : Flight left early.
# Action taken : None 

+-------------+----------+-----------+--------------------+-------+---------+-----------------+--------------------+--------------+-----------+---------------+---------------+--------------------+------------+---------+-------------+----------+-------+-------+---------+--------+------+----------+-------+--------+--------------+-----------------+---------+--------+---------+------------+---+---------+-------+-------------+--------+------------+
|TRANSACTIONID|FLIGHTDATE|AIRLINECODE|         AIRLINENAME|TAILNUM|FLIGHTNUM|ORIGINAIRPORTCODE|     ORIGAIRPORTNAME|ORIGINCITYNAME|ORIGINSTATE|ORIGINSTATENAME|DESTAIRPORTCODE|     DESTAIRPORTNAME|DESTCITYNAME|DESTSTATE|DESTSTATENAME|CRSDEPTIME|DEPTIME|TAXIOUT|WHEELSOFF|WHEELSON|TAXIIN|CRSARRTIME|ARRTIME|ARRDELAY|CRSELAPSEDTIME|ACTUALELAPSEDTIME|CANCELLED|DIVERTED| DISTANCE|distance_mod|bin|start_bin|end_bin|DISTANCEGROUP|DEPDELAY|DEPDELAYGT15|
+-------------+----------+-----------+--------------------+-------+---------+-----------------+---------

In [6]:
from pyspark.sql.functions import min, max
delay_miles.createOrReplaceTempView("delay_miles")

df = spark.sql(sql)



sql = """
select
TRANSACTIONID,AIRLINECODE,AIRLINENAME,TAILNUM,FLIGHTNUM,ORIGINAIRPORTCODE,
ORIGAIRPORTNAME,ORIGINCITYNAME,ORIGINSTATE,ORIGINSTATENAME,DESTAIRPORTCODE,DESTAIRPORTNAME,
DESTCITYNAME,DESTSTATE,DESTSTATENAME,CRSDEPTIME,DEPDELAY,TAXIOUT,WHEELSOFF,WHEELSON,TAXIIN,
CRSARRTIME,ARRDELAY,CRSELAPSEDTIME,ACTUALELAPSEDTIME,CANCELLED,DIVERTED,
DISTANCE, distance_mod ,
bin,start_bin,end_bin,DISTANCEGROUP,
DEPDELAYGT15,
to_date(FLIGHTDATE,'yyyyMMdd') FLIGHTDATE,

case 
    WHEN (length(DEPTIME) = 4 ) THEN concat(left(DEPTIME,2) , ':' , right(DEPTIME,2)) 
    WHEN (length(DEPTIME) = 3 ) THEN concat('0' , left(DEPTIME,1) , ':' , right(DEPTIME,2))
    WHEN (length(DEPTIME) = 2 ) THEN concat('00' ,  ':' , right(DEPTIME,2))
    WHEN (length(DEPTIME) = 1 ) THEN concat('00:0' ,   right(DEPTIME,1))
    ELSE Null 
END as DEPTIME,

case 
    WHEN (length(ARRTIME) = 4 ) THEN concat(left(ARRTIME,2) , ':' , right(ARRTIME,2)) 
    WHEN (length(ARRTIME) = 3 ) THEN concat('0' , left(ARRTIME,1) , ':' , right(ARRTIME,2))
    WHEN (length(ARRTIME) = 2 ) THEN concat('00' ,  ':' , right(ARRTIME,2))
    WHEN (length(ARRTIME) = 1 ) THEN concat('00:0' ,   right(ARRTIME,1))
    ELSE Null 
END as ARRTIME
from delay_miles
"""

#sql = "select length(ARRTIME) from delay_miles "
#" WHEN (length(ARRTIME) = 4) THEN concat (left(ARRTIME,2) , ':' right(ARRTIME,2) )"

set_time = spark.sql (sql)
#set_time.show(5)
set_time.createOrReplaceTempView('set_time')

sql = """
select  
TRANSACTIONID,AIRLINECODE,AIRLINENAME,TAILNUM,FLIGHTNUM,ORIGINAIRPORTCODE,
ORIGAIRPORTNAME,ORIGINCITYNAME,ORIGINSTATE,ORIGINSTATENAME,DESTAIRPORTCODE,DESTAIRPORTNAME,
DESTCITYNAME,DESTSTATE,DESTSTATENAME,CRSDEPTIME,DEPDELAY,TAXIOUT,WHEELSOFF,WHEELSON,TAXIIN,
CRSARRTIME,ARRDELAY,CRSELAPSEDTIME,ACTUALELAPSEDTIME,CANCELLED,DIVERTED,
DISTANCE, distance_mod ,
bin,start_bin,end_bin,DISTANCEGROUP,
DEPDELAYGT15,
to_date(FLIGHTDATE,'yyyyMMdd') FLIGHTDATE,
DEPTIME,ARRTIME,
case 
WHEN left(ARRTIME,2) < left(DEPTIME,2) Then 1  
WHEN left(ARRTIME,2) = left(DEPTIME,2) and (RIGHT(ARRTIME,2) < RIGHT(DEPTIME,2)) Then 1 
WHEN ARRTIME is null or DEPTIME is null Then Null 
ELSE 0  
END as NEXTDAYARR
from set_time

"""



#Assumption # no flight is > 23:59 Minutes 
# null values are kept. If ARRTIME or  DEPTIME is null NEXTDAYARR is set to null 

sql1 = """
select count(*) 
from set_time
where ARRTIME is null 
or DEPTIME is null

"""

NEXTDAYARR = spark.sql (sql)
#test1 = spark.sql (sql1)
NEXTDAYARR.show(2)
#test.filter("NEXTDAYARR = 0 ").count()
NEXTDAYARR.filter("NEXTDAYARR is null ").count()
#test.filter("NEXTDAYARR != 0 ").count()
#min_date = day_check.select(max('FLIGHTDATE'))
#min_date.head()
#day_check = spark.sql (sql1)
#day_check.show(2)

+-------------+-----------+--------------------+-------+---------+-----------------+--------------------+--------------+-----------+---------------+---------------+--------------------+------------+---------+-------------+----------+--------+-------+---------+--------+------+----------+--------+--------------+-----------------+---------+--------+---------+------------+---+---------+-------+-------------+------------+----------+-------+-------+----------+
|TRANSACTIONID|AIRLINECODE|         AIRLINENAME|TAILNUM|FLIGHTNUM|ORIGINAIRPORTCODE|     ORIGAIRPORTNAME|ORIGINCITYNAME|ORIGINSTATE|ORIGINSTATENAME|DESTAIRPORTCODE|     DESTAIRPORTNAME|DESTCITYNAME|DESTSTATE|DESTSTATENAME|CRSDEPTIME|DEPDELAY|TAXIOUT|WHEELSOFF|WHEELSON|TAXIIN|CRSARRTIME|ARRDELAY|CRSELAPSEDTIME|ACTUALELAPSEDTIME|CANCELLED|DIVERTED| DISTANCE|distance_mod|bin|start_bin|end_bin|DISTANCEGROUP|DEPDELAYGT15|FLIGHTDATE|DEPTIME|ARRTIME|NEXTDAYARR|
+-------------+-----------+--------------------+-------+---------+----------------

30366

In [7]:
#33822

In [8]:
NEXTDAYARR.createOrReplaceTempView('NEXTDAYARR')

In [9]:
sql = """
select 
split(AIRLINENAME, ":")[0] AIRLINENAME ,
split(AIRLINENAME, ":")[1] AIRLINECODE 
from NEXTDAYARR
"""

airlinename_sql = """
select  
TRANSACTIONID,TAILNUM,FLIGHTNUM,ORIGINAIRPORTCODE,
ORIGAIRPORTNAME,ORIGINCITYNAME,ORIGINSTATE,ORIGINSTATENAME,DESTAIRPORTCODE,DESTAIRPORTNAME,
DESTCITYNAME,DESTSTATE,DESTSTATENAME,CRSDEPTIME,DEPDELAY,TAXIOUT,WHEELSOFF,WHEELSON,TAXIIN,
CRSARRTIME,ARRDELAY,CRSELAPSEDTIME,ACTUALELAPSEDTIME,CANCELLED,DIVERTED,
DISTANCE, distance_mod ,
bin,start_bin,end_bin,DISTANCEGROUP,
DEPDELAYGT15,
to_date(FLIGHTDATE,'yyyyMMdd') FLIGHTDATE,
DEPTIME,ARRTIME,NEXTDAYARR,
split(AIRLINENAME, ":")[0] AIRLINENAME , split(AIRLINENAME, ":")[1] AIRLINECODE 
from 
NEXTDAYARR
"""

airlinename = spark.sql(airlinename_sql)
df.show(5, False)

+-------------+----------+-----------+------------------------------+-------+---------+-----------------+------------------------------------------------+--------------+-----------+---------------+---------------+-----------------------------------------------+------------+---------+-------------+----------+-------+-------+---------+--------+------+----------+-------+--------+--------------+-----------------+---------+--------+---------+------------+---+---------+-------+-------------+--------+------------+
|TRANSACTIONID|FLIGHTDATE|AIRLINECODE|AIRLINENAME                   |TAILNUM|FLIGHTNUM|ORIGINAIRPORTCODE|ORIGAIRPORTNAME                                 |ORIGINCITYNAME|ORIGINSTATE|ORIGINSTATENAME|DESTAIRPORTCODE|DESTAIRPORTNAME                                |DESTCITYNAME|DESTSTATE|DESTSTATENAME|CRSDEPTIME|DEPTIME|TAXIOUT|WHEELSOFF|WHEELSON|TAXIIN|CRSARRTIME|ARRTIME|ARRDELAY|CRSELAPSEDTIME|ACTUALELAPSEDTIME|CANCELLED|DIVERTED|DISTANCE |distance_mod|bin|start_bin|end_bin|DISTANCEGRO

In [10]:
airlinename.createOrReplaceTempView("airlinename")

orig_dest_cleanupsql = """
select  
TRANSACTIONID,TAILNUM,FLIGHTNUM,ORIGINAIRPORTCODE,
ORIGINCITYNAME,ORIGINSTATE,ORIGINSTATENAME,DESTAIRPORTCODE,
DESTCITYNAME,DESTSTATE,DESTSTATENAME,CRSDEPTIME,DEPDELAY,TAXIOUT,WHEELSOFF,WHEELSON,TAXIIN,
CRSARRTIME,ARRDELAY,CRSELAPSEDTIME,ACTUALELAPSEDTIME,CANCELLED,
DISTANCE, distance_mod ,
bin,start_bin,end_bin,DISTANCEGROUP,
DEPDELAYGT15,
to_date(FLIGHTDATE,'yyyyMMdd') FLIGHTDATE,
DEPTIME,ARRTIME,NEXTDAYARR,
AIRLINECODE,
AIRLINENAME,
split(ORIGAIRPORTNAME, ":")[0] OrigCITYSTATE,
split(ORIGAIRPORTNAME, ":")[1] ORIGAIRPORTNAME,
split(DESTAIRPORTNAME, ":")[0] destCITYSTATE,
split(DESTAIRPORTNAME, ":")[0] DESTAIRPORTNAME,
case 
 WHEN DIVERTED = 1 or DIVERTED = "T" or DIVERTED = "True" Then "True"
 WHEN DIVERTED = 0 or DIVERTED = "F" or DIVERTED = "False" Then "False"
 Else Null 
END as Diverted  
 
from airlinename

"""
final_fact = spark.sql(orig_dest_cleanupsql)
final_fact.show(5)
x=final_fact.count()
xx=final_fact.na.drop()
y=xx.count()
print(x)
print(y)

+-------------+-------+---------+-----------------+--------------+-----------+---------------+---------------+------------+---------+-------------+----------+--------+-------+---------+--------+------+----------+--------+--------------+-----------------+---------+---------+------------+---+---------+-------+-------------+------------+----------+-------+-------+----------+-----------+--------------------+-------------+--------------------+-------------+---------------+--------+
|TRANSACTIONID|TAILNUM|FLIGHTNUM|ORIGINAIRPORTCODE|ORIGINCITYNAME|ORIGINSTATE|ORIGINSTATENAME|DESTAIRPORTCODE|DESTCITYNAME|DESTSTATE|DESTSTATENAME|CRSDEPTIME|DEPDELAY|TAXIOUT|WHEELSOFF|WHEELSON|TAXIIN|CRSARRTIME|ARRDELAY|CRSELAPSEDTIME|ACTUALELAPSEDTIME|CANCELLED| DISTANCE|distance_mod|bin|start_bin|end_bin|DISTANCEGROUP|DEPDELAYGT15|FLIGHTDATE|DEPTIME|ARRTIME|NEXTDAYARR|AIRLINECODE|         AIRLINENAME|OrigCITYSTATE|     ORIGAIRPORTNAME|destCITYSTATE|DESTAIRPORTNAME|Diverted|
+-------------+-------+---------+---

In [11]:
#final_fact.coalesce(1).write\
#      .option("header","true")\
#      .option("sep",",")\
#      .csv("FACT_FLIGHTS.csv")


In [12]:
final_fact.createOrReplaceTempView("final_fact")
dim_airline_sql = """
select distinct airlinecode , AIRLINENAME
from final_fact
"""

dim_airline =spark.sql(dim_airline_sql)
dim_airline.count()
dim_airline.show(5)

+-----------+--------------------+
|airlinecode|         AIRLINENAME|
+-----------+--------------------+
|         B6|     JetBlue Airways|
|         CO|Continental Air L...|
|         EV|ExpressJet Airlin...|
|         F9|Frontier Airlines...|
|         DH|    Independence Air|
+-----------+--------------------+
only showing top 5 rows



In [13]:
dim_airportcode_sql1 = """


select distinct ORIGINAIRPORTCODE , ORIGAIRPORTNAME
from final_fact

"""

dim_airportcode_sql2 = """


select distinct DESTAIRPORTCODE , DESTAIRPORTNAME
from final_fact

"""
dim_airportcode1 = spark.sql(dim_airportcode_sql1)
dim_airportcode1.createOrReplaceTempView("dim_airportcode1")


dim_airportcode2 = spark.sql(dim_airportcode_sql2)
dim_airportcode2.createOrReplaceTempView("dim_airportcode2")

dim_airportcode_sql = """
select ORIGINAIRPORTCODE, ORIGAIRPORTNAME
from 
dim_airportcode1 A 
FUll outer join 
dim_airportcode2 B 
ON A.ORIGINAIRPORTCODE = B.DESTAIRPORTCODE
"""



dim_airportcode =spark.sql(dim_airportcode_sql)


dim_airportcode.count()
#dim_airportcode.show(10)

365

In [14]:
dim_city1 = """
select distinct ORIGINCITYNAME,ORIGINSTATE , ORIGINSTATENAME
from 
final_fact

"""

dim_city1 =spark.sql(dim_city1)


dim_city1.count()
#dim_city1.show(10)
dim_city1.createOrReplaceTempView("dim_city1")


dim_city2 = """
select distinct DESTCITYNAME,DESTSTATE, DESTSTATENAME
from 
final_fact """
dim_city2 =spark.sql(dim_city2)


#dim_city2.count()
#dim_city2.show(10)
dim_city2.createOrReplaceTempView("dim_city2")

dim_city = """
select ORIGINCITYNAME, ORIGINSTATE, ORIGINSTATENAME
from 
dim_city1 A 
Full outer join 
dim_city2 B 
ON A.ORIGINSTATE = B.DESTSTATE
"""



dim_city3 =spark.sql(dim_city)


dim_city3.count()
#dim_city3.show(10)
dim_city3.createOrReplaceTempView("dim_city3")

dim_city = """
select distinct ORIGINCITYNAME as cityname , ORIGINSTATE as statecode from dim_city3

"""
dim_city =spark.sql(dim_city)
dim_city.show()


dim_state = """

select distinct ORIGINSTATE as statecode  , ORIGINSTATENAME as STATENAME from dim_city3
"""

dim_state =spark.sql(dim_state)
dim_state.show()

+--------------------+---------+
|            cityname|statecode|
+--------------------+---------+
|           Asheville|       NC|
|   Dallas/Fort Worth|       TX|
|              Monroe|       LA|
|         Minneapolis|       MN|
|Newport News/Will...|       VA|
|            Lewiston|       ID|
|     Bismarck/Mandan|       ND|
|       Crescent City|       CA|
|             Ontario|       CA|
|               Butte|       MT|
|          Burlington|       VT|
|        Grand Island|       NE|
|                Rota|       TT|
|             Lansing|       MI|
|    Charlotte Amalie|       VI|
|Pasco/Kennewick/R...|       WA|
|           St. Cloud|       MN|
|               Flint|       MI|
|              Bangor|       ME|
|        Fayetteville|       AR|
+--------------------+---------+
only showing top 20 rows

+---------+--------------------+
|statecode|           STATENAME|
+---------+--------------------+
|       VI| U.S. Virgin Islands|
|       MT|             Montana|
|       NC|      

In [19]:
dim_airline_sql = "select distinct AIRLINECODE, AIRLINENAME from final_fact"

dim_airline = spark.sql(dim_airline_sql)
dim_airline.show(3)

dim_airline.coalesce(1).write\
      .option("header","true")\
      .option("sep",",")\
      .csv("dim_airline5.csv")

+-----------+--------------------+
|AIRLINECODE|         AIRLINENAME|
+-----------+--------------------+
|         B6|     JetBlue Airways|
|         CO|Continental Air L...|
|         EV|ExpressJet Airlin...|
+-----------+--------------------+
only showing top 3 rows



In [15]:
fact_flights.coalesce(1).write\
      .option("header","true")\
      .option("sep",",")\
      .csv("FACT_FLIGHTS5.csv")

dim_airportcode.coalesce(1).write\
      .option("header","true")\
      .option("sep",",")\
      .csv("dim_airportcode5.csv")

dim_city.coalesce(1).write\
      .option("header","true")\
      .option("sep",",")\
      .csv("dim_city5.csv")
dim_state.coalesce(1).write\
      .option("header","true")\
      .option("sep",",")\
      .csv("dim_statea5.csv")

dim_airline.coalesce(1).write\
      .option("header","true")\
      .option("sep",",")\
      .csv("dim_airline5.csv")

In [16]:
final_fact.printSchema()

root
 |-- TRANSACTIONID: string (nullable = true)
 |-- TAILNUM: string (nullable = true)
 |-- FLIGHTNUM: string (nullable = true)
 |-- ORIGINAIRPORTCODE: string (nullable = true)
 |-- ORIGINCITYNAME: string (nullable = true)
 |-- ORIGINSTATE: string (nullable = true)
 |-- ORIGINSTATENAME: string (nullable = true)
 |-- DESTAIRPORTCODE: string (nullable = true)
 |-- DESTCITYNAME: string (nullable = true)
 |-- DESTSTATE: string (nullable = true)
 |-- DESTSTATENAME: string (nullable = true)
 |-- CRSDEPTIME: string (nullable = true)
 |-- DEPDELAY: string (nullable = true)
 |-- TAXIOUT: string (nullable = true)
 |-- WHEELSOFF: string (nullable = true)
 |-- WHEELSON: string (nullable = true)
 |-- TAXIIN: string (nullable = true)
 |-- CRSARRTIME: string (nullable = true)
 |-- ARRDELAY: string (nullable = true)
 |-- CRSELAPSEDTIME: string (nullable = true)
 |-- ACTUALELAPSEDTIME: string (nullable = true)
 |-- CANCELLED: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- dista

dim_city.show(10)

dim_city.coalesce(1).write\
      .option("header","true")\
      .option("sep",",")\
      .csv("dim_city2.csv")

In [17]:
sql = "select NEXTDAYARR from final_fact where NEXTDAYARR not in (1 , 0) "
ss =spark.sql(sql)
ss.count()

0